# Setup

In [1]:
import sys
sys.path.append("..")
import os
os.chdir("./..")
from utils import get_opensearch_client
os.chdir("./lab")

os_client = get_opensearch_client()

# Opensearch Indices

In [7]:
response = os_client.cat.indices(format='json')

for index in response:
    print(index['index'])


test-index
.opensearch-observability
.plugins-ml-config
security-auditlog-2024.03.17
.opendistro_security


# Get docs from a index

In [11]:
def get_all_docs(index_name):
    # Execute the search query to get the first 1000 documents
    response = os_client.search(
        index=index_name,
        body={"query": {"match_all": {}}},
        size=1000
    )

    # Iterate through the hits and print document details
    for doc in response['hits']['hits']:
        print(doc['_id'], doc['_source'])

# Example usage
index_name = 'test-index'
get_all_docs(index_name)

1 {'action': 'index', 'title': 'El barco velero', 'content': 'Content of document 1'}
2 {'action': 'index', 'title': 'Hawai', 'content': 'Content of document 2'}
3 {'action': 'index', 'title': 'Document 3 Updated', 'content': 'Updated content of document 3'}
4 {'action': 'index', 'title': 'Document 4 Updated', 'content': 'Updated content of document 3'}


In [3]:
def get_all_docs(index_name):
    # Execute the search query to get the first 1000 documents
    response = os_client.search(
        index=index_name,
        body={"query": {"match_all": {}}},
        size=1000
    )

    # Iterate through the hits and print document details
    for doc in response['hits']['hits']:
        print(doc['_id'], doc['_source'])

# Example usage
index_name = 'data'
get_all_docs(index_name)

../data/Víctor Gutiérrez Tovar.pdf {'file_name': 'Víctor Gutiérrez Tovar', 'text': "Víctor Gutiérrez Tovar\nFinal year Software Engineering student, specializing in\nData Engineering and Full Stack development.\nTeamWork \uf4cf\nEngagement \uf483\nProfessionalism \uf8f7\n\uf4e7+34 656593435\n\uf84cvictorgt2406@gmail.com\n\uf3eePortfolio\n\uf472victor-gutierrez-tovar\n\uf3edvictorgt2406\nSkills\nBackend\nCloud\nComputer science\nData engineer\nData processing\nDatabases\nDevOps\nFrontend\nInterpersonal skills\nLanguage\nProgramming languages\nSearch engines\nSoftware\nWeb\nExperiences\nEducation\nTechnical details. This Curriculum, along with the portfolio, is made with ReactJs using the Vite development service and hosted with GitHub actions. Bootstrap 5 is\nbeing used for the styling of this website along with a CSS to customize it. For more details, visit the portfolio's GitHub.\nGitHub/portfolio\nInternship | Data Engineer\nTabacalera\n\uf3e7Madrid, Spain\n\uf1e802/2024 - 03/2023\nI